# Dynamic Programming

- **Created by Andrés Segura Tinoco**
- **Created on Jan 26, 2020**

Dynamic programming is both a mathematical optimization method and a computer programming method. In both contexts it refers to simplifying a complicated problem by breaking it down into simpler sub-problems in a recursive manner <a href="#link_one">[1]</a>.

In [1]:
# Load the Python libraries
import timeit
import math

## 1. Binomial Coefficient

In [2]:
# Example values
n = 20
k = 10

### Simple approach

In [3]:
# The recursive natural solution
def C(n, k):
    if k == 0 or k == n:
        return 1
    return C(n - 1, k - 1) + C(n - 1, k)

In [4]:
start_time = timeit.default_timer()
print(C(n, k))
print('>> elapsed time', (timeit.default_timer() - start_time) * 1000, 'ms')

184756
>> elapsed time 157.7332 ms


### With Dynamic Programming

In [5]:
# Solution with dynamic programming (supported by a table)
def C2(n, k):
    c = 0
    v = [1] * (k + 1)
    
    for i in range(n + 1):
        for j in range(k, 0, -1):
            if j < i:
                v[j] = v[j - 1] + v[j]
    
    return v[k]

In [6]:
start_time = timeit.default_timer()
print(C2(n, k))
print('>> elapsed time', (timeit.default_timer() - start_time) * 1000, 'ms')

184756
>> elapsed time 1.0717000000000088 ms


With computational complexity of $ \Theta(nk) $ and a space complexity of $ \Theta(k) $.

## 2. World Championship Problem

In [7]:
# Example values
n = 10
p = 0.55
q = 1 - p

### Simple approach

In [8]:
# The recursive natural solution
def WCP(i, j):
    if i == 0:
        return 1
    elif j == 0:
        return 0
    return p * WCP(i - 1, j) + q * WCP(i, j - 1)

In [9]:
start_time = timeit.default_timer()
print(WCP(n, n))
print('>> elapsed time', (timeit.default_timer() - start_time) * 1000, 'ms')

0.6710359124216079
>> elapsed time 116.54350000000002 ms


### With Dynamic Programming

In [10]:
# Solution with dynamic programming (supported by a table)
def WCP2(n, p):
    n = n + 1
    q = 1 - p
    prob = [[0] * n for i in range(n)]
    
    for s in range(n):
        prob[0][s] = 1
        for k in range(1, s):
            prob[k][s - k] = p * prob[k - 1][s - k] + q * prob[k][s - k - 1]
    
    for s in range(1, n):
        for k in range(0, n - s):
            prob[s + k][n - k - 1] = p * prob[s + k - 1][n - k - 1] + q * prob[s + k][n - k - 2]
    
    return prob[n - 1][n - 1]

In [11]:
start_time = timeit.default_timer()
print(WCP2(n, p))
print('>> elapsed time', (timeit.default_timer() - start_time) * 1000, 'ms')

0.6710359124216079
>> elapsed time 0.9335999999999789 ms


With computational complexity of $ O(n^2) $ and a space complexity of $ \Theta(n^2) $.

## 3. Coin Change Problem

The **coin-change problem** or change-making problem addresses the question of finding the minimum number of coins (of certain denominations) that add up to a given amount of money. It is a special case of the integer knapsack problem, and has applications wider than just currency <a href="#link_two">[2]</a>.

#### Returns all possible combinations of coins change:

In [12]:
def coin_change(N, d):
    n = len(d)
    c = [[0] * (N + 1) for i in range(n)]
    
    for i in range(0, n):
        for j in range(1, N + 1):
            if i == 0 and j < d[i]:
                c[i][j] = math.inf
            elif i == 0:
                c[i][j] = 1 + c[0][j - d[0]]
            elif j < d[i]:
                c[i][j] = c[i - 1][j]
            else:
                c[i][j] = min(c[i - 1][j], 1 + c[i][j - d[i]])
    
    return c

In [13]:
# Example
N = 8
d = [1, 4, 6]
coins = coin_change(N, d)
coins

[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [0, 1, 2, 3, 1, 2, 3, 4, 2],
 [0, 1, 2, 3, 1, 2, 1, 2, 2]]

With computational complexity of $ \Theta(nN) $ and a space complexity of $ \Theta(n(N + 1)) $.

#### Calculate the list of coins needed to give change

In [14]:
def get_coins_list(c, d, N):
    coins_list = []
    i = len(d) - 1
    j = N
    
    while i > -1 and j > -1:
        if i - 1 >= 0 and c[i][j] == c[i - 1][j]:
            i = i - 1
        elif j - d[i] >= 0 and c[i][j] == 1 + c[i][j - d[i]]:
            coins_list.append(d[i])
            j = j - d[i]
        else:
            break
    return coins_list

In [15]:
for j in range(1, N + 1):
    print(j, '->', get_coins_list(coins, d, j))

1 -> [1]
2 -> [1, 1]
3 -> [1, 1, 1]
4 -> [4]
5 -> [4, 1]
6 -> [6]
7 -> [6, 1]
8 -> [4, 4]


With computational complexity of $ \Theta(n + c[n, N]) $ and a space complexity of $ \Theta(n(N + 1)) $.

## Reference

<a name='link_one' href='https://en.wikipedia.org/wiki/Dynamic_programming' target='_blank' >[1]</a> Wikipedia - Dynamic Programming.  
<a name='link_two' href='https://en.wikipedia.org/wiki/Change-making_problem' target='_blank' >[2]</a> Wikipedia - Change-making problem.  

---
<a href="https://ansegura7.github.io/Algorithms/">« Home</a>